In [10]:
!pip install langchain>=0.2.10
!pip install langchain-community>=0.2.10
!pip install langchain-openai>=0.1.7
!pip install langchain-experimental>=0.0.64
!pip install chromadb>=0.5.5
!pip install tiktoken>=0.7.0
!pip install pypdf
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 65.4 MB/s eta 0:00:00


In [2]:
# 🔐 Set your Azure OpenAI credentials directly here

AZURE_API_KEY = "4Nqwnlcz9GrGBss7JS3ex5wbXigVzBSXHcpXdIw5hXekF88Kgzr7JQQJ99BGACYeBjFXJ3w3AAABACOGX0Ld"
AZURE_API_BASE = "https://oai9999999999999999.openai.azure.com/"
AZURE_EMBEDDING_DEPLOYMENT = "text-embedding-ada-002"
AZURE_LLM_DEPLOYMENT = "gpt-4o-mini"

# ✅ Confirm
print("✅ Azure credentials configured.")


✅ Azure credentials configured.


In [4]:
from langchain_community.document_loaders import PyPDFLoader

# 📄 Provide path to your PDF file
pdf_path = "INTRODUCTION TO EV.pdf"  # Change if needed
loader = PyPDFLoader(pdf_path)
pages = loader.load()

print(f"✅ Loaded {len(pages)} pages from PDF.")


✅ Loaded 9 pages from PDF.


In [5]:
from langchain_openai import AzureOpenAIEmbeddings
from langchain_experimental.text_splitter import SemanticChunker

embeddings = AzureOpenAIEmbeddings(
    azure_deployment=AZURE_EMBEDDING_DEPLOYMENT,
    api_key=AZURE_API_KEY,
    azure_endpoint=AZURE_API_BASE,
    api_version="2024-02-15-preview",  # works well with langchain-openai
)

# You can tune breakpoint behavior; defaults are fine to start.
# Options: breakpoint_threshold_type={"percentile"|"standard_deviation"},
# breakpoint_threshold_amount (e.g., 95 for percentile)
chunker = SemanticChunker(
    embeddings=embeddings,
    breakpoint_threshold_type="percentile",
    breakpoint_threshold_amount=95,
)

semantic_chunks = chunker.split_documents(pages)
print(f"✅ Created {len(semantic_chunks)} semantic chunks.")


✅ Created 17 semantic chunks.


In [8]:
# Print each chunk separately
for i, chunk in enumerate(semantic_chunks, start=1):
    print(f"--- Chunk {i} ---")
    print(chunk.page_content)
    print()

--- Chunk 1 ---
EV OVERVIEW 
An electric vehicle is powered by a battery 
- Edrive and it does not need any type of
fossil fuel. While conventional vehicles use
an internal 
combustion engine and gaso -
line or diesel as fuel, electric vehicles use
electrical energy 
that is stored inside the
high voltage 
battery. As a result, electric
vehicles run 
on electricity, they are eco-
friendly - they do not require fuel and are
zero emission vehicles. Review and Characteristics 
This is an electric vehicle. Some of the ve-
hicle’s systems operate differently and  
have different operating characteristics  
than vehicles equipped with an internal  
combustion engine. It is important to care-
fully read the entire Owner’s Manual for  
this reason. This Vehicle uses two types of battery sys-
tems- a high voltage system in which a  
high voltage battery powers the inverter  
and electric motor which in turn propel the 
vehicle and a low voltage system in which 
a 12-volt battery provides power 

In [11]:
from langchain_community.vectorstores import FAISS

def build_faiss(chunks, embeddings):
    db = FAISS.from_documents(documents=chunks, embedding=embeddings)
    db.save_local("faiss_index")  # Optional: saves index for later use
    return db

db = build_faiss(semantic_chunks, embeddings)

In [12]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    deployment_name=AZURE_LLM_DEPLOYMENT,
    api_key=AZURE_API_KEY,
    azure_endpoint=AZURE_API_BASE,
    api_version="2024-02-15-preview",
    temperature=0,
)
print("✅ Azure Chat LLM ready.")


✅ Azure Chat LLM ready.


In [13]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=db.as_retriever(search_kwargs={"k": 4}),
    chain_type="stuff",
)

q = "Explain how the regenerative braking system works in an EV"
result = qa.invoke({"query": q})

print("Q:", q)
print("A:", result["result"])


Q: Explain how the regenerative braking system works in an EV
A: The regenerative braking system in an electric vehicle (EV) works by converting the vehicle's kinetic energy back into electrical energy during braking or coasting. When the driver applies the brakes or releases the accelerator, the electric motor that normally propels the vehicle acts as a generator. 

As the vehicle slows down, the motor reverses its function and uses the motion of the vehicle to generate electricity. This electricity is then used to recharge the high voltage battery, effectively capturing energy that would otherwise be lost as heat in traditional braking systems. This process not only helps to extend the driving range of the EV by replenishing the battery but also contributes to overall energy efficiency.


# 🔹 RAG Questions from EV Document

1. **What is the main difference between an electric vehicle and a conventional internal combustion engine vehicle?**  

2. **Explain how the regenerative braking system works in an EV.**  

3. **What are the key components of the 3-in-1 unit in an electric vehicle?**  

4. **Describe the role of the high-voltage battery and how it interacts with the low-voltage battery.**  

5. **What safety precautions should be taken when handling the high-voltage system of an EV?**  

6. **In what situations is the high-voltage system still considered safe, even if water enters the vehicle?**  

7. **List the abbreviations EV, HV battery, LV battery, OBC, and PDU along with their meanings.**  

8. **What general warnings are provided about the lithium-ion high-voltage battery in EVs?**  

9. **Why are electric vehicles considered environmentally friendly compared to conventional vehicles?**  

10. **What are some important warnings, cautions, and notes highlighted in the EV manual?**

